# Coleta de dados
Nesta seção iremos coletar os dados por meio de web scraping e apis de outros sites

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
prefix = 'https://www.transfermarkt.com'
def getSoup(url):
    res = requests.get(url, headers = headers)
    return BeautifulSoup(res.content, 'html.parser')


### Função para realizar o web scraping das tableas de transferências no transfermarkt

In [17]:
def getClubUrl(url):
    url = url.replace('/transfers', '')  # Remove a substring '/transfers'
    match = re.search(r'/verein/(\d+)', url)
    if(not match):
        return "invalid";
    verein_id = match.group(1)  # Obtém o número após "/verein/"
    new_url = match.string[:match.start()] + \
        '/startseite' '/verein/' + verein_id
    return new_url


def eurValue(string):
    # Expressão regular para verificar se começa com '€' e termina com 'm' ou 'k'
    padrao = r'^€.*[mk]$'
    if re.match(padrao, string):
        return True
    else:
        return False


def eurToInt(string):
    valor_str = re.sub(r'[€mk]', '', string)  # Remove '€', 'm' e 'k' da string
    # Substitui a vírgula pelo ponto como separador decimal e converte para float
    valor_float = float(valor_str.replace(',', '.'))
    if 'm' in string:
        valor_int = int(valor_float * 1000000)  # Converte para milhões
    elif 'k' in string:
        valor_int = int(valor_float * 1000)  # Converte para milhares
    else:
        valor_int = int(valor_float)
    return valor_int


def getTransfersFromUrl(url, year):
    transferTable = []
    page = getSoup(url.format(year))
    tables = page.find_all('div', {'class': 'box'})
    tables = tables[3:]
    for table in tables:
        currTeamUrl = getClubUrl(table.find('a').get('href'))
        if(currTeamUrl == "invalid"):
            break;
        currTeam = table.find('a').get('title')
        arrivalsAndDepartures = table.find_all('table')

        arrivals = arrivalsAndDepartures[0].find_all('tr', recursive=True)
        arrivals = arrivals[1:]
        for transfer in arrivals:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            playerCountry = transfer[2].find('img').get('alt')
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubLeftUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubJoinedUrl = currTeamUrl
            clubLeft = transfer[6].find('a').get('title')
            clubJoined = currTeam
            teamTransferType = "out"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

        departures = arrivalsAndDepartures[1].find_all(
            'tr', recursive=True)
        departures = departures[1:]
        for transfer in departures:
            transfer = transfer.find_all('td')
            player = transfer[0].find('a')
            if isinstance(player, type(None)):
                continue
            playerName = player.text
            playerLink = player.get('href')
            playerAge = transfer[1].text
            playerPos = transfer[4].text
            playerCountry = transfer[2].find('img').get('alt')
            transferFee = transfer[8].text
            if (not eurValue(transferFee)):
                continue
            if (not eurValue(transfer[5].text)):
                playerMarketValue = 0
            else:
                playerMarketValue = eurToInt(transfer[5].text)
            transferFee = eurToInt(transferFee)
            if (transferFee < 10):
                continue
            clubJoinedUrl = getClubUrl(transfer[6].find('a').get('href'))
            clubLeftUrl = currTeamUrl
            clubLeft = currTeam
            clubJoined = transfer[6].find('a').get('title')
            teamTransferType = "in"

            transferTable.append([playerName, playerLink, playerAge, playerPos, playerCountry,
                                  playerMarketValue, transferFee, clubLeft, clubLeftUrl, clubJoined, clubJoinedUrl, teamTransferType])

    columns = ['playerName', 'playerLink', 'playerAge', 'playerPos', 'playerCountry',
               'playerMarketValue', 'transferFee', 'clubLeft', 'clubLeftUrl', 'clubJoined', 'clubJoinedUrl', 'teamTransferType']
    final_df = pd.DataFrame(transferTable, columns=columns)
    final_df['season'] = year
    final_df['playerAge'] = final_df['playerAge'].apply(
        lambda x: re.sub(r'\D+', '', str(x)))
    final_df['playerAge'] = final_df['playerAge'].astype(int)
    final_df['valuation'] = final_df['playerMarketValue'] - final_df['transferFee']
    return final_df


### Função para fazer o web scraping das tabelas de transferências de um determinado período e concatenar elas

In [18]:
def getTransfersInRange(begin, end, url):
    curr_df = getTransfersFromUrl(url, begin)
    for i in range(begin+1, end+1):
        curr_df = pd.concat(
            [curr_df, getTransfersFromUrl(url, i)]).reset_index(drop=True)
    return curr_df

### Obtendo dados das transferências para um range de temporadas da Ligue One(demora 3 a 6 segundos por temporada, depende da internet)

In [19]:
l1 = r'https://www.transfermarkt.com/ligue-1/transfers/wettbewerb/FR1/plus/?saison_id={}&s_w=&leihe=0&intern=0'
l1_transfers = getTransfersInRange(2000, 2023, l1)
l1_transfers

,playerName,playerLink,playerAge,playerPos,playerCountry,playerMarketValue,transferFee,clubLeft,clubLeftUrl,clubJoined,clubJoinedUrl,teamTransferType,season,valuation
0,Aleksandr Panov,/aleksandr-panov/profil/spieler/17800,24,CF,Russia,0,4700000,Zenit St. Petersburg,/zenit-st-petersburg/startseite/verein/964,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,out,2000,-4700000
1,Maksym Levitsky,/maksym-levitsky/profil/spieler/41048,27,GK,Ukraine,0,850000,Chernomorets Novorossijsk,/chernomorets-novorossijsk/startseite/verein/5287,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,out,2000,-850000
2,Vincent Péricard,/vincent-pericard/profil/spieler/3970,17,CF,France,0,200000,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,Juventus FC,/juventus-turin/startseite/verein/506,in,2000,-200000
3,Mehdi Méniri,/mehdi-meniri/profil/spieler/12116,23,CB,Algeria,0,1220000,AS Nancy-Lorraine,/as-nancy-lorraine/startseite/verein/1159,ESTAC Troyes,/es-troyes-ac/startseite/verein/1095,out,2000,-1220000
4,Mikkel Beck,/mikkel-beck/profil/spieler/5650,27,CF,Denmark,0,350000,Derby County,/derby-county/startseite/verein/22,LOSC Lille,/losc-lille/startseite/verein/1082,out,2000,-350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2960,Ibrahim Cissoko,/ibrahim-cissoko/profil/spieler/575693,20,LW,Netherlands,2500000,3000000,NEC Nijmegen,/nec-nijmegen/startseite/verein/467,FC Toulouse,/fc-toulouse/startseite/verein/415,out,2023,-500000
2961,Mahdi Camara,/mahdi-camara/profil/spieler/324795,25,CM,France,3000000,3000000,AS Saint-Étienne,/as-saint-etienne/startseite/verein/618,Stade Brestois 29,/stade-brest-29/startseite/verein/3911,out,2023,0
2962,Mostafa Mohamed,/mostafa-mohamed/profil/spieler/462348,25,CF,Egypt,6000000,5750000,Galatasaray,/galatasaray-istanbul/startseite/verein/141,FC Nantes,/fc-nantes/startseite/verein/995,out,2023,250000
2963,Andy Delort,/andy-delort/profil/spieler/122797,31,CF,Algeria,7000000,5000000,OGC Nice,/ogc-nizza/startseite/verein/417,FC Nantes,/fc-nantes/startseite/verein/995,out,2023,2000000


In [20]:
l1_transfers = l1_transfers[l1_transfers['playerMarketValue'] != 0]
l1_transfers = l1_transfers.drop_duplicates(subset=['playerLink', 'playerAge', 'transferFee'])
l1_transfers.to_csv('data/l1_transfers.csv', index=False)

OSError: ignored